vgg16 Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Convert the labels to categorical format
num_classes = 2
train_labels_categorical = to_categorical(train_labels, num_classes)
val_labels_categorical = to_categorical(val_labels, num_classes)
test_labels_categorical = to_categorical(test_labels, num_classes)

# Resize the input images to match VGG16's expected input shape
target_size = (256, 256)
train_images_resized = tf.image.resize(train_images, target_size)
val_images_resized = tf.image.resize(val_images, target_size)
test_images_resized = tf.image.resize(test_images, target_size)

# Create ImageDataGenerator instances for image preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Prepare the data generators
train_generator = train_datagen.flow(train_images_resized, train_labels_categorical, batch_size=25)
val_generator = val_datagen.flow(val_images_resized, val_labels_categorical, batch_size=25)
test_generator = test_datagen.flow(test_images_resized, test_labels_categorical, batch_size=25)

# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Create a new model
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for 20 epochs
history = model.fit(train_generator, epochs=20,
                    validation_data=val_generator, verbose=1)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)

# Evaluate the model on validation data
validation_loss, validation_acc = model.evaluate(val_generator)

# Print the test and validation accuracy
print('Test accuracy:', test_acc)
print('Validation accuracy:', validation_acc)